# PreProcess

## 목적
- faze 노말라이제이션 정보 추가
- everyone-profile.npz -> everyone-faze-profile.npz

## 수행 일지

### 1차
- Normalized Camera 정보 Faze 원본과 동일하게 유지
- 병렬 변환 코드 (병렬 처리에 버그가 있는지 머신이 죽은건지 도널드가 변환 3일 째에 먹통됨)
  - 마지막에 처리된 프로파일들은 확인차 다시 변환을 해서 해쉬 비교 필요
  

In [1]:
from normalizer import *

# Configuration

In [2]:
SUPPLE_PATH = '/Volumes/SSD3/faze-resources/gs.h5'
FACE_MODEL_PATH = '/Volumes/SSD3/faze-resources/sfm_face_coordinates.npy'
NPZ_BASE_PATH = '/Volumes/SSD3/everyone-npz'
OUT_DIR_PATH = '/Volumes/SSD3/faze-recode'

# Check Supplementary 
- 총 포함 프로파일 아이디 1366개

In [3]:
def profile_to_idx():
    pid_to_idx = {}
    
    with h5py.File(SUPPLE_PATH, 'r') as f:
        profiles = f.keys()
        
        idx = 0
        for pid in profiles:
            pid_to_idx[pid] = idx
            idx = idx + 1

#         num_entries = next(iter(group.values())).shape[0]
#         print("profile-{} num: {}".format(person, num_entries))

#         for i in range(0, 5): # range(num_entries):
#             fx, fy, cx, cy = group['camera_parameters'][i, :]
#             image_path = '%s' % (group['file_name'][i].decode('utf-8'))
#             rvec = group['head_pose'][i, :3].reshape(3, 1)
#             tvec = group['head_pose'][i, 3:].reshape(3, 1)
#             distor = group['distortion_parameters'][i, :]
#             gaze_t = group['3d_gaze_target'][i, :].reshape(3, 1)
#             # print(rvec) # print(tvec) # print(distor)  
    return pid_to_idx

In [6]:
idxs = profile_to_idx()
print(idxs["00010"])

4


# Prepare Processor

In [7]:
num_profiles = 1366
normalizer = FazeNormalizer(OUT_DIR_PATH, NPZ_BASE_PATH, SUPPLE_PATH, FACE_MODEL_PATH)

In [8]:
# missing_list = ["00539","00865","01177","01528","01941","01943","02394","02925","03491","03492","03495","03498","03501","03502","03523"]
# for pid in missing_list:
#     idx = idxs[pid]
#     normalizer.run_preprocess(idx_from=idx, idx_to=idx+1)

In [9]:
normalizer.run_preprocess(idx_from=4, idx_to=5)

>>> [4] Process profile 00010



# Sanity Check

In [ ]:
def show_image(img, cmap=None, extent=None, title=None, fig_size=(10, 10)):
    if cmap is None:
        cmap = select_colormap_by_shape(img.shape)
    if extent is None:
        extent = get_image_extent(img)

    img = squeeze_if_gray(img)
    fig = plt.figure(figsize=fig_size)

    plt.axis('off')
    if title is not None:
        plt.title(title)
    ax = fig.add_subplot(111)
    ax.imshow(img, cmap=cmap, interpolation='none', extent=extent)
    plt.axis('image')

    return ax

def sanity_check_npz_recode(profile_id, idx, root_path="/Volumes/SSD3/everyone-faze/"):
    npz = np.load(os.path.join(root_path, "profile-recode-{}.npz".format(profile_id)), allow_pickle=True)
    print(npz.files)
                  
    face = npz['norm_face'][idx]
    img = byte_arr_to_img(face)
    print(img.shape)
    show_image(img)

# 좌우 눈 영역만으로 representation 학습을 하면 잘 될까?
- Faze 원문은 일단 전체 이미지가 가로로 길쭉한 양안 이미지를 사용함
- 아마도 실험해보고 크게 차이 없어서 이렇게 바꿨을 지도?